<a href="https://colab.research.google.com/github/varshini03/VeriSyn-DeepfakeDetector/blob/main/fakeavfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: display content in mounted drive

from google.colab import drive
drive.mount('/content/drive')

# Display the contents of the mounted drive
!ls '/content/drive/My Drive'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'Colab Notebooks'	        df_trial.ipynb	 FakeAVCeleb_v1.2       LIPINC-main
'Copy of DeepfakeFinal.ipynb'   FakeAVCeleb	 FakeAVCeleb_v1.2.zip   RESUME_SREE_VARSHINI.pdf


In [ ]:
!ls '/content/drive/My Drive/FakeAVCeleb_v1.2/RealVideo-RealAudio/African/men/id00076'

00109.mp4  00109_roi.mp4  00109_text.txt  00109.wav


In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import librosa
from torchvision import transforms
from tensorflow.keras.applications import EfficientNetB0
import os
from tqdm import tqdm
import cv2
import pickle

EXTRACT FEATURES

In [ ]:
def load_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
        else:
            break
    cap.release()
    return np.array(frames)

def load_audio(audio_path):
    audio, _ = librosa.load(audio_path, sr=16000)
    return audio

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),  # Resize to 224x224 for EfficientNet-b0
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize to ImageNet mean and std
])

# Load the pre-trained EfficientNetB0 model without the top classification layer
model = EfficientNetB0(include_top=False, weights='imagenet')

def extract_visual_feature(model, video_path, audio_path):
    frames = load_video(video_path)
    transformed_frames = []
    for frame in frames:
        transformed_frame = transform(frame)
        transformed_frames.append(transformed_frame)
    frames_tensor = torch.stack(transformed_frames)
    frames_tensor = frames_tensor.permute(0, 2, 3, 1)

    audio = load_audio(audio_path)
    audio = torch.tensor(audio)

    # Define the frame length and hop length
    frame_length = 0.025  # Frame length in seconds
    hop_length = 0.01  # Hop length in seconds

    # Convert frame length and hop length to samples
    frame_length_samples = int(frame_length * 16000)  # Use 16000 Hz as the sample rate
    hop_length_samples = int(hop_length * 16000)  # Use 16000 Hz as the sample rate

    # Extract MFCCs for each frame
    mfccs = librosa.feature.mfcc(y=audio.cpu().numpy(), sr=16000, n_mfcc=13, hop_length=hop_length_samples, n_fft=frame_length_samples)
    mfccs = torch.tensor(mfccs)

    # Ensure that the number of frames in the video matches the number of audio frames
    if frames_tensor.shape[0] > mfccs.shape[1]:
        frames_tensor = frames_tensor[:mfccs.shape[1]]
    elif frames_tensor.shape[0] < mfccs.shape[1]:
        mfccs = mfccs[:, :frames_tensor.shape[0]]

    with torch.no_grad():
        feature_vid = model(frames_tensor)
        f = feature_vid.cpu().numpy()

    return f, mfccs.cpu().numpy()

16705208/16705208 [==============================] - 0s 0us/step


In [2]:
def load_video(video_path, num_frames=25):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames >= num_frames:
        indices = np.round(np.linspace(0, total_frames - 1, num_frames)).astype(int)
        for i in range(total_frames):
            ret, frame = cap.read()
            if i in indices:
                frames.append(frame)
    else:
        for i in range(total_frames):
            ret, frame = cap.read()
            frames.append(frame)
        last_frame = frames[-1]
        for _ in range(num_frames - total_frames):
            frames.append(last_frame)

    cap.release()

    return np.array(frames)

def load_audio(audio_path):
    audio, _ = librosa.load(audio_path, sr=16000)
    return audio

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),  # Resize to 224x224 for EfficientNet-b0
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize to ImageNet mean and std
])

# Load the pre-trained EfficientNetB0 model without the top classification layer
model = EfficientNetB0(include_top=False, weights='imagenet')

def extract_visual_feature(model, video_path, audio_path):
    frames = load_video(video_path)
    transformed_frames = []
    for frame in frames:
        transformed_frame = transform(frame)
        transformed_frames.append(transformed_frame)
    frames_tensor = torch.stack(transformed_frames)
    frames_tensor = frames_tensor.permute(0, 2, 3, 1)

    audio = load_audio(audio_path)
    audio = torch.tensor(audio)

    # Define the frame length and hop length
    frame_length = 0.025  # Frame length in seconds
    hop_length = 0.01  # Hop length in seconds

    # Convert frame length and hop length to samples
    frame_length_samples = int(frame_length * 16000)  # Use 16000 Hz as the sample rate
    hop_length_samples = int(hop_length * 16000)  # Use 16000 Hz as the sample rate

    # Extract MFCCs for each frame
    mfccs = librosa.feature.mfcc(y=audio.cpu().numpy(), sr=16000, n_mfcc=13, hop_length=hop_length_samples, n_fft=frame_length_samples)
    mfccs = torch.tensor(mfccs)

    # Ensure that the number of frames in the video matches the number of audio frames
    if frames_tensor.shape[0] > mfccs.shape[1]:
        frames_tensor = frames_tensor[:mfccs.shape[1]]
    elif frames_tensor.shape[0] < mfccs.shape[1]:
        mfccs = mfccs[:, :frames_tensor.shape[0]]

    with torch.no_grad():
        feature_vid = model(frames_tensor)
        f = feature_vid.cpu().numpy()

    return f, mfccs.cpu().numpy()

16705208/16705208 [==============================] - 0s 0us/step


In [ ]:
print(features,labels)
m = len(features[0])
for i in range(2,100):
  m = min(m,len(features[i]))
print(m)

#features size range - 99-330 for videos

[array([[ -0.21553567,  -0.18586574,  -0.27406168, ...,  -2.1479392 ,
         -2.4439921 ,  -6.1380043 ],
       [ -0.21546914,  -0.18583944,  -0.2740537 , ..., -15.189792  ,
         -2.4071355 ,  -5.828635  ],
       [ -0.21551767,  -0.18586908,  -0.27406144, ...,  -8.657143  ,
          4.2383466 ,  -8.901952  ],
       ...,
       [ -0.21754555,  -0.18661118,  -0.27354473, ..., -21.95414   ,
          1.2516623 ,   1.9154036 ],
       [ -0.21719456,  -0.1859487 ,  -0.2738531 , ..., -13.920692  ,
         -1.7427645 ,  -4.8499384 ],
       [ -0.2173499 ,  -0.18592262,  -0.2737749 , ...,  -9.577986  ,
          3.811412  ,  -9.51008   ]], dtype=float32), array([[ -0.21770109,  -0.18762659,  -0.27298993, ...,  26.717709  ,
        -10.423573  ,  -8.19199   ],
       [ -0.21776144,  -0.18757893,  -0.27302977, ...,  29.012604  ,
         -5.971885  , -15.933817  ],
       [ -0.21787204,  -0.18746176,  -0.2730927 , ...,   7.0961704 ,
         -4.6348042 , -12.784122  ],
       ...,
    

In [ ]:
import pickle

# Save the arrays
with open('/content/drive/My Drive/FakeAVCeleb_v1.2/rvra_features.pkl', 'wb') as f1:
    pickle.dump(features, f1)

with open('/content/drive/My Drive/FakeAVCeleb_v1.2/rvra_labels.pkl', 'wb') as f2:
    pickle.dump(labels, f2)

In [ ]:
import pickle

# Load the arrays back into memory
with open('/content/drive/My Drive/FakeAVCeleb_v1.2/rvra_features.pkl', 'rb') as f1:
    features = pickle.load(f1)

with open('/content/drive/My Drive/FakeAVCeleb_v1.2/rvra_labels.pkl', 'rb') as f2:
    labels = pickle.load(f2)

# Print the arrays to check their content
print(features,labels)
m = len(features[0])
for i in range(2,100):
  m = min(m,len(features[i]))
print(m)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/FakeAVCeleb_v1.2/RealVideo-RealAudio/rvra_features.pkl'

In [ ]:
from sklearn.decomposition import PCA
# Initialize an empty list for storing the features and labels
features = []
labels = []

# Directory walk
PATH_TO_DIRECTORY = '/content/drive/My Drive/FakeAVCeleb_v1.2/'
categories = ['rvra']#, 'rvfa', 'fvra', 'fvfa']

for category in categories:
    if category == 'rvra':
        input_root = os.path.join(PATH_TO_DIRECTORY, 'RealVideo-RealAudio')
        label = 0
    else:
        if category == 'rvfa':
            input_root = os.path.join(PATH_TO_DIRECTORY, 'RealVideo-FakeAudio')
        elif category == 'fvra':
            input_root = os.path.join(PATH_TO_DIRECTORY, 'FakeVideo-RealAudio')
        elif category == 'fvfa':
            input_root = os.path.join(PATH_TO_DIRECTORY, 'FakeVideo-FakeAudio')
        label = 1

    to_iterate = list(os.walk(input_root))
    for root, dirs, files in tqdm(to_iterate, total=len(to_iterate)):
        for file in files:
            if file.endswith('_roi.mp4'):
                input_video_path = os.path.join(root, file)
                input_audio_path = os.path.join(root, file[:-8] + '.wav')

                # Extract the features
                f, mfccs = extract_visual_feature(model, input_video_path, input_audio_path)
                # Reduce dimensions of f
                f = f.reshape((f.shape[0], -1))  # Reshape f to 2D

                # Concatenate the features and add them to the list
                features.append(np.concatenate([f, mfccs.T], axis=-1))

                # Add the label to the list
                labels.append(label)




100%|██████████| 104/104 [40:03<00:00, 23.11s/it]


In [ ]:
print(len(features))
print(len(labels))
print(f.shape,mfccs.shape)

100
100
(105, 62720) (13, 105)


In [ ]:
import pickle

# Save the arrays
with open('/content/drive/My Drive/FakeAVCeleb_v1.2/rvra_features.pkl', 'wb') as f1:
    pickle.dump(features, f1)

with open('/content/drive/My Drive/FakeAVCeleb_v1.2/rvra_labels.pkl', 'wb') as f2:
    pickle.dump(labels, f2)

In [4]:
from sklearn.decomposition import PCA
# Initialize an empty list for storing the features and labels
features = []
labels = []

# Directory walk
PATH_TO_DIRECTORY = '/content/drive/My Drive/FakeAVCeleb_v1.2/'
categories = ['rvra']#, 'rvfa', 'fvra', 'fvfa']

for category in categories:
    if category == 'rvra':
        input_root = os.path.join(PATH_TO_DIRECTORY, 'RealVideo-RealAudio')
        label = 0
    else:
        if category == 'rvfa':
            input_root = os.path.join(PATH_TO_DIRECTORY, 'RealVideo-FakeAudio')
        elif category == 'fvra':
            input_root = os.path.join(PATH_TO_DIRECTORY, 'FakeVideo-RealAudio')
        elif category == 'fvfa':
            input_root = os.path.join(PATH_TO_DIRECTORY, 'FakeVideo-FakeAudio')
        label = 1

    to_iterate = list(os.walk(input_root))
    for root, dirs, files in tqdm(to_iterate, total=len(to_iterate)):
        for file in files:
            if file.endswith('_roi.mp4'):
                input_video_path = os.path.join(root, file)
                input_audio_path = os.path.join(root, file[:-8] + '.wav')

                # Extract the features
                f, mfccs = extract_visual_feature(model, input_video_path, input_audio_path)
                # Reduce dimensions of f
                f = f.reshape((f.shape[0], -1))  # Reshape f to 2D

                # Concatenate the features and add them to the list
                features.append(np.concatenate([f, mfccs.T], axis=-1))

                # Add the label to the list
                labels.append(label)


  0%|          | 0/104 [00:00<?, ?it/s]WARNING:tensorflow:From <ipython-input-2-b4a4e6da3e48>:70: _EagerTensorBase.cpu (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.identity with explicit device placement instead.
100%|██████████| 104/104 [02:34<00:00,  1.49s/it]


In [9]:
print(len(features))
print(len(labels))
print(f.shape,mfccs.shape)
m = len(features[0])
for i in range(2,100):
  m = max(m,len(features[i]))
print(m)

100
100
(25, 62720) (13, 25)
25


In [10]:
# save as .npy file
features = np.array(features)
labels = np.array(labels)
np.save('/content/drive/My Drive/FakeAVCeleb_v1.2/rvra_features.npy', features)
np.save('/content/drive/My Drive/FakeAVCeleb_v1.2/rvra_labels.npy', labels)

In [11]:
# load from .npy file
features1 = np.load('/content/drive/My Drive/FakeAVCeleb_v1.2/rvra_features.npy')
labels1 = np.load('/content/drive/My Drive/FakeAVCeleb_v1.2/rvra_labels.npy')

print(features1,labels1)

[[[ -0.21553579  -0.18586577  -0.27406168 ...  -2.1479392   -2.4439921
    -6.1380043 ]
  [ -0.21717408  -0.18545711  -0.2738458  ... -15.189792    -2.4071355
    -5.828635  ]
  [ -0.21702646  -0.18577763  -0.27365077 ...  -8.657143     4.2383466
    -8.901952  ]
  ...
  [ -0.21769507  -0.1866018   -0.2733513  ... -19.404577    -6.9667172
    -3.2468657 ]
  [ -0.21702123  -0.18655275  -0.2735521  ... -12.477049    -3.766316
   -22.216078  ]
  [ -0.21734981  -0.18592286  -0.2737749  ...  -8.677618   -13.669761
   -14.910971  ]]

 [[ -0.2177013   -0.18762615  -0.27298996 ...  26.717709   -10.423573
    -8.19199   ]
  [ -0.2177653   -0.1874231   -0.27306172 ...  29.012604    -5.971885
   -15.933817  ]
  [ -0.21873234  -0.1877257   -0.27307072 ...   7.0961704   -4.6348042
   -12.784122  ]
  ...
  [ -0.21810095  -0.1873014   -0.27304578 ...   4.3850465  -16.282429
    31.894953  ]
  [ -0.21784416  -0.18727642  -0.27310327 ...  15.468962    -7.090953
    11.709892  ]
  [ -0.21794495  -0.1873

In [12]:
from sklearn.decomposition import PCA
# Initialize an empty list for storing the features and labels
features = []
labels = []

# Directory walk
PATH_TO_DIRECTORY = '/content/drive/My Drive/FakeAVCeleb_v1.2/'
categories = ['rvfa']#, 'rvfa', 'fvra', 'fvfa']

for category in categories:
    if category == 'rvra':
        input_root = os.path.join(PATH_TO_DIRECTORY, 'RealVideo-RealAudio')
        label = 0
    else:
        if category == 'rvfa':
            input_root = os.path.join(PATH_TO_DIRECTORY, 'RealVideo-FakeAudio')
        elif category == 'fvra':
            input_root = os.path.join(PATH_TO_DIRECTORY, 'FakeVideo-RealAudio')
        elif category == 'fvfa':
            input_root = os.path.join(PATH_TO_DIRECTORY, 'FakeVideo-FakeAudio')
        label = 1

    to_iterate = list(os.walk(input_root))
    for root, dirs, files in tqdm(to_iterate, total=len(to_iterate)):
        for file in files:
            if file.endswith('_roi.mp4'):
                input_video_path = os.path.join(root, file)
                input_audio_path = os.path.join(root, file[:-8] + '.wav')

                # Extract the features
                f, mfccs = extract_visual_feature(model, input_video_path, input_audio_path)
                # Reduce dimensions of f
                f = f.reshape((f.shape[0], -1))  # Reshape f to 2D

                # Concatenate the features and add them to the list
                features.append(np.concatenate([f, mfccs.T], axis=-1))

                # Add the label to the list
                labels.append(label)


100%|██████████| 104/104 [02:11<00:00,  1.27s/it]


In [13]:
# save as .npy file
features = np.array(features)
labels = np.array(labels)
np.save('/content/drive/My Drive/FakeAVCeleb_v1.2/rvfa_features.npy', features)
np.save('/content/drive/My Drive/FakeAVCeleb_v1.2/rvfa_labels.npy', labels)

In [14]:
print(len(features))
print(len(labels))
print(f.shape,mfccs.shape)

100
100
(25, 62720) (13, 25)


In [15]:
from sklearn.decomposition import PCA
# Initialize an empty list for storing the features and labels
features = []
labels = []

# Directory walk
PATH_TO_DIRECTORY = '/content/drive/My Drive/FakeAVCeleb_v1.2/'
categories = ['fvra']#, 'rvfa', 'fvra', 'fvfa']

for category in categories:
    if category == 'rvra':
        input_root = os.path.join(PATH_TO_DIRECTORY, 'RealVideo-RealAudio')
        label = 0
    else:
        if category == 'rvfa':
            input_root = os.path.join(PATH_TO_DIRECTORY, 'RealVideo-FakeAudio')
        elif category == 'fvra':
            input_root = os.path.join(PATH_TO_DIRECTORY, 'FakeVideo-RealAudio')
        elif category == 'fvfa':
            input_root = os.path.join(PATH_TO_DIRECTORY, 'FakeVideo-FakeAudio')
        label = 1

    to_iterate = list(os.walk(input_root))
    for root, dirs, files in tqdm(to_iterate, total=len(to_iterate)):
        for file in files:
            if file.endswith('_roi.mp4'):
                input_video_path = os.path.join(root, file)
                input_audio_path = os.path.join(root, file[:-8] + '.wav')

                # Extract the features
                f, mfccs = extract_visual_feature(model, input_video_path, input_audio_path)
                # Reduce dimensions of f
                f = f.reshape((f.shape[0], -1))  # Reshape f to 2D

                # Concatenate the features and add them to the list
                features.append(np.concatenate([f, mfccs.T], axis=-1))

                # Add the label to the list
                labels.append(label)


100%|██████████| 104/104 [41:45<00:00, 24.09s/it]


In [20]:
# save as .npy file
features = np.array(features)
labels = np.array(labels)
np.save('/content/drive/My Drive/FakeAVCeleb_v1.2/fvra_features.npy', features)
np.save('/content/drive/My Drive/FakeAVCeleb_v1.2/fvra_labels.npy', labels)

In [17]:
print(len(features))
print(len(labels))
print(f.shape,mfccs.shape)

1850
1850
(25, 62720) (13, 25)


In [19]:
# load from .npy file
features1 = np.load('/content/drive/My Drive/FakeAVCeleb_v1.2/fvra_features.npy')
labels1 = np.load('/content/drive/My Drive/FakeAVCeleb_v1.2/fvra_labels.npy')

print(features1,labels1)

[[[ -0.21728443  -0.19065866  -0.27191392 ... -10.287678    -0.14026248
    -3.7251348 ]
  [ -0.21817164  -0.18941824  -0.2710543  ... -20.641937     8.833809
   -25.49377   ]
  [ -0.21714823  -0.18814705  -0.27183825 ... -29.714558     8.040483
   -24.030323  ]
  ...
  [ -0.21669826  -0.19112398  -0.27103022 ...  -1.3236544    5.4852724
    -1.7872348 ]
  [ -0.215456    -0.18848853  -0.272555   ...  -7.2609434    3.038221
    -4.0209837 ]
  [ -0.21739426  -0.18750349  -0.27221847 ... -12.209359    -2.3402333
    -9.747793  ]]

 [[ -0.21479568  -0.1903256   -0.27273187 ...  -0.15268016  -5.0622764
     3.0150015 ]
  [ -0.21757075  -0.18862225  -0.2713449  ...   8.961884   -12.80596
    -9.166618  ]
  [ -0.21736713  -0.18775465  -0.27207556 ...   9.325798   -11.102903
   -13.603825  ]
  ...
  [ -0.2164418   -0.18834154  -0.2720047  ...  -0.11887956  -4.178855
     8.68233   ]
  [ -0.21776704  -0.18647313  -0.27305818 ...  10.518613     6.9844866
    -9.319471  ]
  [ -0.21486965  -0.1868

In [ ]:
from sklearn.decomposition import PCA
# Initialize an empty list for storing the features and labels
features = []
labels = []

# Directory walk
PATH_TO_DIRECTORY = '/content/drive/My Drive/FakeAVCeleb_v1.2/'
categories = ['fvfa']#, 'rvfa', 'fvra', 'fvfa']

for category in categories:
    if category == 'rvra':
        input_root = os.path.join(PATH_TO_DIRECTORY, 'RealVideo-RealAudio')
        label = 0
    else:
        if category == 'rvfa':
            input_root = os.path.join(PATH_TO_DIRECTORY, 'RealVideo-FakeAudio')
        elif category == 'fvra':
            input_root = os.path.join(PATH_TO_DIRECTORY, 'FakeVideo-RealAudio')
        elif category == 'fvfa':
            input_root = os.path.join(PATH_TO_DIRECTORY, 'FakeVideo-FakeAudio')
        label = 1

    to_iterate = list(os.walk(input_root))
    for root, dirs, files in tqdm(to_iterate, total=len(to_iterate)):
        for file in files:
            if file.endswith('_roi.mp4'):
                input_video_path = os.path.join(root, file)
                input_audio_path = os.path.join(root, file[:-8] + '.wav')

                # Extract the features
                f, mfccs = extract_visual_feature(model, input_video_path, input_audio_path)
                # Reduce dimensions of f
                f = f.reshape((f.shape[0], -1))  # Reshape f to 2D

                # Concatenate the features and add them to the list
                features.append(np.concatenate([f, mfccs.T], axis=-1))

                # Add the label to the list
                labels.append(label)


In [ ]:
# save as .npy file
features = np.array(features)
labels = np.array(labels)
np.save('/content/drive/My Drive/FakeAVCeleb_v1.2/fvfa_features.npy', features)
np.save('/content/drive/My Drive/FakeAVCeleb_v1.2/fvfa_labels.npy', labels)

In [6]:
import pickle
# Load the data
with open('/content/drive/My Drive/rvra_labels.pkl', 'rb') as f1:
    rvra_labels = pickle.load(f1)

In [7]:
print(rvra_labels)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [8]:
import pickle
# Load the data
with open('/content/drive/My Drive/rvfa_labels.pkl', 'rb') as f1:
    rvfa_labels = pickle.load(f1)
print(rvfa_labels)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [10]:
import pickle
# Load the data
with open('/content/drive/My Drive/fvra_labels.pkl', 'rb') as f1:
    fvra_labels = pickle.load(f1)
print(fvra_labels)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [1]:
import pickle
# Load the data
with open('/content/drive/My Drive/rvfa_features.pkl', 'rb') as f1:
    rvfa_features = pickle.load(f1)
print(rvfa_features)

[array([[ -0.21757846,  -0.18817836,  -0.27361137, ...,   0.        ,
          0.        ,   0.        ],
       [ -0.21767429,  -0.18852158,  -0.273781  , ...,   0.        ,
          0.        ,   0.        ],
       [ -0.21781369,  -0.18974544,  -0.27391756, ...,  -2.834507  ,
          4.229444  ,  12.290577  ],
       ...,
       [ -0.22062829,  -0.19068582,  -0.27508786, ...,  -6.202835  ,
        -15.194981  ,  -5.188288  ],
       [ -0.22063142,  -0.19068605,  -0.27508888, ...,  -1.3320472 ,
         -2.8863783 , -10.311728  ],
       [ -0.22060816,  -0.19069353,  -0.27509156, ...,  10.129547  ,
         12.232566  ,   1.727511  ]], dtype=float32), array([[ -0.21261396,  -0.18827991,  -0.2729016 , ...,   0.        ,
          0.        ,   0.        ],
       [ -0.20688987,  -0.18659398,  -0.27140927, ...,   0.        ,
          0.        ,   0.        ],
       [ -0.20589463,  -0.18582557,  -0.27092418, ...,   0.        ,
          0.        ,   0.        ],
       ...,
    

In [6]:
print(rvfa_features[20].shape)
print(rvfa_features[30].shape)
print(rvfa_features[50].shape)
print(rvfa_features[60].shape)

(183, 62733)
(104, 62733)
(105, 62733)
(110, 62733)


In [2]:
from numpy import savetxt
savetxt('data.csv', rvfa_features, delimiter=',')

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (100,) + inhomogeneous part.

In [ ]:
print(rvra_features[-1]s.shape)
print(rvfa_features[-1]shape)
print(fvra_features[-1].shape)
print()
print(rvra_feature[0]s.shape)
print(rvfa_features[0]shape)
print(fvra_features[0].shape)

In [13]:
import pickle
# Load the data
with open('/content/drive/My Drive/fvra_features.pkl', 'rb') as f1:
    fvra_features = pickle.load(f1)
print(fvra_features)

UnpicklingError: pickle data was truncated

In [ ]:
import pickle
# Load the data
with open('/content/drive/My Drive/fvra_features.pkl', 'rb') as f1:
    fvra_features = pickle.load(f1)
print(fvra_features)

NORMALISE EXTRACTED FEATURES

In [2]:
# load from .npy file
features1 = np.load('/content/drive/My Drive/FakeAVCeleb_v1.2/rvra_features.npy')
labels1 = np.load('/content/drive/My Drive/FakeAVCeleb_v1.2/rvra_labels.npy')

print(features1,labels1)

[[[ -0.21553579  -0.18586577  -0.27406168 ...  -2.1479392   -2.4439921
    -6.1380043 ]
  [ -0.21717408  -0.18545711  -0.2738458  ... -15.189792    -2.4071355
    -5.828635  ]
  [ -0.21702646  -0.18577763  -0.27365077 ...  -8.657143     4.2383466
    -8.901952  ]
  ...
  [ -0.21769507  -0.1866018   -0.2733513  ... -19.404577    -6.9667172
    -3.2468657 ]
  [ -0.21702123  -0.18655275  -0.2735521  ... -12.477049    -3.766316
   -22.216078  ]
  [ -0.21734981  -0.18592286  -0.2737749  ...  -8.677618   -13.669761
   -14.910971  ]]

 [[ -0.2177013   -0.18762615  -0.27298996 ...  26.717709   -10.423573
    -8.19199   ]
  [ -0.2177653   -0.1874231   -0.27306172 ...  29.012604    -5.971885
   -15.933817  ]
  [ -0.21873234  -0.1877257   -0.27307072 ...   7.0961704   -4.6348042
   -12.784122  ]
  ...
  [ -0.21810095  -0.1873014   -0.27304578 ...   4.3850465  -16.282429
    31.894953  ]
  [ -0.21784416  -0.18727642  -0.27310327 ...  15.468962    -7.090953
    11.709892  ]
  [ -0.21794495  -0.1873

In [4]:
# load from .npy file
features2 = np.load('/content/drive/My Drive/FakeAVCeleb_v1.2/rvfa_features.npy')
labels2 = np.load('/content/drive/My Drive/FakeAVCeleb_v1.2/rvfa_labels.npy')

print(features2,labels2)

[[[ -0.21757841  -0.18817836  -0.2736114  ...   0.           0.
     0.        ]
  [ -0.21772477  -0.19064349  -0.27374625 ...   0.           0.
     0.        ]
  [ -0.21816446  -0.19171053  -0.2737062  ...  -2.834505     4.229445
    12.29057   ]
  ...
  [ -0.24251084  -0.22180621  -0.26278153 ...  -7.030238   -10.220714
   -12.40023   ]
  [ -0.21909036  -0.18876904  -0.2750663  ...   0.66008306 -16.7221
   -12.24515   ]
  [ -0.2206081   -0.19069345  -0.27509156 ...   2.3798811  -15.372645
   -10.739261  ]]

 [[ -0.21261404  -0.18827984  -0.27290156 ...   0.           0.
     0.        ]
  [ -0.20411351  -0.18468861  -0.2708256  ...   0.           0.
     0.        ]
  [ -0.21148792  -0.18783541  -0.27277687 ...   0.           0.
     0.        ]
  ...
  [ -0.21206558  -0.18955891  -0.27319682 ... -14.838175     0.12730569
    -6.717118  ]
  [ -0.2124617   -0.18903728  -0.27297857 ... -31.65903    -18.527908
   -13.68387   ]
  [ -0.21206735  -0.18907695  -0.27310067 ... -25.28479    

In [5]:
# load from .npy file
features3 = np.load('/content/drive/My Drive/FakeAVCeleb_v1.2/fvra_features.npy')
labels3 = np.load('/content/drive/My Drive/FakeAVCeleb_v1.2/fvra_labels.npy')

print(features3,labels3)

[[[ -0.21728443  -0.19065866  -0.27191392 ... -10.287678    -0.14026248
    -3.7251348 ]
  [ -0.21817164  -0.18941824  -0.2710543  ... -20.641937     8.833809
   -25.49377   ]
  [ -0.21714823  -0.18814705  -0.27183825 ... -29.714558     8.040483
   -24.030323  ]
  ...
  [ -0.21669826  -0.19112398  -0.27103022 ...  -1.3236544    5.4852724
    -1.7872348 ]
  [ -0.215456    -0.18848853  -0.272555   ...  -7.2609434    3.038221
    -4.0209837 ]
  [ -0.21739426  -0.18750349  -0.27221847 ... -12.209359    -2.3402333
    -9.747793  ]]

 [[ -0.21479568  -0.1903256   -0.27273187 ...  -0.15268016  -5.0622764
     3.0150015 ]
  [ -0.21757075  -0.18862225  -0.2713449  ...   8.961884   -12.80596
    -9.166618  ]
  [ -0.21736713  -0.18775465  -0.27207556 ...   9.325798   -11.102903
   -13.603825  ]
  ...
  [ -0.2164418   -0.18834154  -0.2720047  ...  -0.11887956  -4.178855
     8.68233   ]
  [ -0.21776704  -0.18647313  -0.27305818 ...  10.518613     6.9844866
    -9.319471  ]
  [ -0.21486965  -0.1868

In [14]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
print(features1.shape)
features_2d = features1.reshape(features1[0], -1)
features_shape = features.shape
features_2d = features.reshape(features_shape[0], -1)

scaler = StandardScaler()
features_2d_norm = scaler.fit_transform(features_2d)

features_norm = features_2d_norm.reshape(features_shape)

(100, 25, 62733)


TypeError: only integer scalar arrays can be converted to a scalar index

In [16]:
from sklearn.preprocessing import MinMaxScaler

f_shape = features1.shape

# Create a scaler
scaler = MinMaxScaler(feature_range=(-1, 1))

# Normalize f
f_2d = features1.reshape(f_shape[0], -1)
f_2d_norm = scaler.fit_transform(f_2d)
f_norm = f_2d_norm.reshape(f_shape)

print(f_norm)

[[[-0.17952728  0.7270508  -0.90842056 ... -0.13993445  0.2564427
   -0.1856009 ]
  [ 0.17400551  0.9173269  -0.58120346 ... -0.27951315  0.11598359
   -0.0171487 ]
  [ 0.07205105  0.80475616 -0.9153271  ... -0.16625229  0.02992421
   -0.01649746]
  ...
  [ 0.5176773   0.7520256  -0.6635895  ... -0.44608897 -0.00442326
   -0.00634465]
  [ 0.3361473   0.05308914 -0.5646744  ... -0.31095365  0.04466207
   -0.5722969 ]
  [-0.17365074 -0.06860256 -0.5658531  ... -0.19434725 -0.15073305
   -0.4530009 ]]

 [[-0.33986473  0.41796112 -0.8489647  ...  1.0000001  -0.02247542
   -0.26992002]
  [ 0.13742256  0.7708683  -0.47097778 ...  0.9999999   0.01531498
   -0.30908704]
  [-0.01046085  0.6414585  -0.89472866 ...  0.3298393  -0.23360857
   -0.12144038]
  ...
  [ 0.4900093   0.7141342  -0.6282215  ...  0.22133371 -0.2974981
    0.9999999 ]
  [ 0.24599075 -0.01585579 -0.4993286  ...  0.43664286 -0.06366073
    0.58629775]
  [-0.20521069 -0.17724705 -0.47608566 ...  0.62693274  0.02174278
    0.14

In [18]:
features1 = f_norm

In [19]:
print(features1)

[[[-0.17952728  0.7270508  -0.90842056 ... -0.13993445  0.2564427
   -0.1856009 ]
  [ 0.17400551  0.9173269  -0.58120346 ... -0.27951315  0.11598359
   -0.0171487 ]
  [ 0.07205105  0.80475616 -0.9153271  ... -0.16625229  0.02992421
   -0.01649746]
  ...
  [ 0.5176773   0.7520256  -0.6635895  ... -0.44608897 -0.00442326
   -0.00634465]
  [ 0.3361473   0.05308914 -0.5646744  ... -0.31095365  0.04466207
   -0.5722969 ]
  [-0.17365074 -0.06860256 -0.5658531  ... -0.19434725 -0.15073305
   -0.4530009 ]]

 [[-0.33986473  0.41796112 -0.8489647  ...  1.0000001  -0.02247542
   -0.26992002]
  [ 0.13742256  0.7708683  -0.47097778 ...  0.9999999   0.01531498
   -0.30908704]
  [-0.01046085  0.6414585  -0.89472866 ...  0.3298393  -0.23360857
   -0.12144038]
  ...
  [ 0.4900093   0.7141342  -0.6282215  ...  0.22133371 -0.2974981
    0.9999999 ]
  [ 0.24599075 -0.01585579 -0.4993286  ...  0.43664286 -0.06366073
    0.58629775]
  [-0.20521069 -0.17724705 -0.47608566 ...  0.62693274  0.02174278
    0.14

In [20]:
print(features1.shape)

(100, 25, 62733)


In [23]:
# load from .npy file
fc = np.load('/content/drive/My Drive/FakeAVCeleb_v1.2/rvra_features.npy')
lc = np.load('/content/drive/My Drive/FakeAVCeleb_v1.2/rvra_labels.npy')

print(fc.shape)
print(fc)

(100, 25, 62733)
[[[ -0.21553579  -0.18586577  -0.27406168 ...  -2.1479392   -2.4439921
    -6.1380043 ]
  [ -0.21717408  -0.18545711  -0.2738458  ... -15.189792    -2.4071355
    -5.828635  ]
  [ -0.21702646  -0.18577763  -0.27365077 ...  -8.657143     4.2383466
    -8.901952  ]
  ...
  [ -0.21769507  -0.1866018   -0.2733513  ... -19.404577    -6.9667172
    -3.2468657 ]
  [ -0.21702123  -0.18655275  -0.2735521  ... -12.477049    -3.766316
   -22.216078  ]
  [ -0.21734981  -0.18592286  -0.2737749  ...  -8.677618   -13.669761
   -14.910971  ]]

 [[ -0.2177013   -0.18762615  -0.27298996 ...  26.717709   -10.423573
    -8.19199   ]
  [ -0.2177653   -0.1874231   -0.27306172 ...  29.012604    -5.971885
   -15.933817  ]
  [ -0.21873234  -0.1877257   -0.27307072 ...   7.0961704   -4.6348042
   -12.784122  ]
  ...
  [ -0.21810095  -0.1873014   -0.27304578 ...   4.3850465  -16.282429
    31.894953  ]
  [ -0.21784416  -0.18727642  -0.27310327 ...  15.468962    -7.090953
    11.709892  ]
  [ -0.

In [25]:
# save as .npy file
features = np.array(features1)
print(features.shape)
np.save('/content/drive/My Drive/FakeAVCeleb_v1.2/rvra_features.npy', features)

(100, 25, 62733)


In [27]:
# load from .npy file
fc = np.load('/content/drive/My Drive/FakeAVCeleb_v1.2/rvra_features.npy')

print(fc)

[[[-0.17952728  0.7270508  -0.90842056 ... -0.13993445  0.2564427
   -0.1856009 ]
  [ 0.17400551  0.9173269  -0.58120346 ... -0.27951315  0.11598359
   -0.0171487 ]
  [ 0.07205105  0.80475616 -0.9153271  ... -0.16625229  0.02992421
   -0.01649746]
  ...
  [ 0.5176773   0.7520256  -0.6635895  ... -0.44608897 -0.00442326
   -0.00634465]
  [ 0.3361473   0.05308914 -0.5646744  ... -0.31095365  0.04466207
   -0.5722969 ]
  [-0.17365074 -0.06860256 -0.5658531  ... -0.19434725 -0.15073305
   -0.4530009 ]]

 [[-0.33986473  0.41796112 -0.8489647  ...  1.0000001  -0.02247542
   -0.26992002]
  [ 0.13742256  0.7708683  -0.47097778 ...  0.9999999   0.01531498
   -0.30908704]
  [-0.01046085  0.6414585  -0.89472866 ...  0.3298393  -0.23360857
   -0.12144038]
  ...
  [ 0.4900093   0.7141342  -0.6282215  ...  0.22133371 -0.2974981
    0.9999999 ]
  [ 0.24599075 -0.01585579 -0.4993286  ...  0.43664286 -0.06366073
    0.58629775]
  [-0.20521069 -0.17724705 -0.47608566 ...  0.62693274  0.02174278
    0.14

In [28]:
f_shape2 = features2.shape

f_2d2 = features2.reshape(f_shape[0], -1)
f_2d_norm2 = scaler.fit_transform(f_2d2)
f_norm2 = f_2d_norm2.reshape(f_shape2)

print(f_norm2)

[[[-2.88436890e-01  2.84343719e-01 -8.65087509e-01 ... -1.00000000e+00
   -1.00000000e+00 -1.00000000e+00]
  [ 1.76734924e-02  3.52600098e-01 -5.71769714e-01 ... -1.00000000e+00
   -1.00000000e+00 -1.00000000e+00]
  [-2.04038620e-02  2.83365250e-01 -8.75297546e-01 ... -1.81770518e-01
    6.03618920e-02  4.91243720e-01]
  ...
  [-1.00000000e+00 -1.00000000e+00  5.97433090e-01 ... -3.35397005e-01
   -3.54240477e-01 -2.32526511e-01]
  [ 4.24141884e-01 -2.41369247e-01 -7.90744781e-01 ...  1.22896813e-01
   -5.55469036e-01 -1.99319899e-01]
  [-3.16061974e-01 -4.35498238e-01 -6.96739197e-01 ...  2.72763968e-01
   -5.35468817e-01 -1.87190086e-01]]

 [[ 1.05125427e-01  2.66181946e-01 -8.19835663e-01 ... -1.00000000e+00
   -1.00000000e+00 -1.00000000e+00]
  [ 1.00000095e+00  8.88113022e-01 -1.38248444e-01 ... -1.00000000e+00
   -1.00000000e+00 -1.00000000e+00]
  [ 3.58452797e-01  6.18160248e-01 -8.31769943e-01 ...  4.22954559e-02
   -2.69483924e-01 -5.18760681e-01]
  ...
  [ 6.47973061e-01  6.6

In [29]:
# load from .npy file
fc2 = np.load('/content/drive/My Drive/FakeAVCeleb_v1.2/rvfa_features.npy')
lc2 = np.load('/content/drive/My Drive/FakeAVCeleb_v1.2/rvfa_labels.npy')

print(fc2.shape)
print(fc2)

(100, 25, 62733)
[[[ -0.21757841  -0.18817836  -0.2736114  ...   0.           0.
     0.        ]
  [ -0.21772477  -0.19064349  -0.27374625 ...   0.           0.
     0.        ]
  [ -0.21816446  -0.19171053  -0.2737062  ...  -2.834505     4.229445
    12.29057   ]
  ...
  [ -0.24251084  -0.22180621  -0.26278153 ...  -7.030238   -10.220714
   -12.40023   ]
  [ -0.21909036  -0.18876904  -0.2750663  ...   0.66008306 -16.7221
   -12.24515   ]
  [ -0.2206081   -0.19069345  -0.27509156 ...   2.3798811  -15.372645
   -10.739261  ]]

 [[ -0.21261404  -0.18827984  -0.27290156 ...   0.           0.
     0.        ]
  [ -0.20411351  -0.18468861  -0.2708256  ...   0.           0.
     0.        ]
  [ -0.21148792  -0.18783541  -0.27277687 ...   0.           0.
     0.        ]
  ...
  [ -0.21206558  -0.18955891  -0.27319682 ... -14.838175     0.12730569
    -6.717118  ]
  [ -0.2124617   -0.18903728  -0.27297857 ... -31.65903    -18.527908
   -13.68387   ]
  [ -0.21206735  -0.18907695  -0.27310067 

In [33]:
features2 = f_norm2
# save as .npy file
features = np.array(features2)
print(features.shape)
np.save('/content/drive/My Drive/FakeAVCeleb_v1.2/rvfa_features.npy', features)

(100, 25, 62733)


In [32]:
f_shape3 = features3.shape

f_2d3 = features3.reshape(f_shape3[0], -1)
f_2d_norm3 = scaler.fit_transform(f_2d3)
f_norm3 = f_2d_norm3.reshape(f_shape3)

print(f_norm3)

[[[-0.45003796  0.21230125 -0.7818556  ... -0.4030553   0.21940936
    0.04989316]
  [ 0.12524414  0.44643116 -0.46552277 ... -0.43733472  0.36734313
   -0.47842765]
  [ 0.05436325  0.59883976 -0.77150726 ... -0.74561745  0.11390406
   -0.3464042 ]
  ...
  [ 0.28573418  0.75104284 -0.82909775 ...  0.1036524   0.256652
    0.04553828]
  [ 0.5361204   0.42758083 -0.62916756 ... -0.10568325  0.17749491
    0.07214069]
  [ 0.36461258  0.55227804 -0.7549715  ... -0.2619584   0.12811372
   -0.17834693]]

 [[-0.2675724   0.24370384 -0.81840897 ... -0.02392392  0.062493
    0.3163016 ]
  [ 0.16543579  0.49206257 -0.49119377 ...  0.41959792 -0.2139622
   -0.04089205]
  [ 0.04389286  0.6236124  -0.7809038  ...  0.4295082  -0.41664124
   -0.08105597]
  ...
  [ 0.2956524   0.8069246  -0.8554554  ...  0.1359233  -0.01874784
    0.34221733]
  [ 0.44482708  0.51072264 -0.66960335 ...  0.34325662  0.29602918
   -0.09613782]
  [ 0.43701887  0.5701275  -0.7963629  ...  0.3306499   0.17437617
   -0.18402

In [35]:
features3 = f_norm3
# save as .npy file
features = np.array(features3)
print(features.shape)
np.save('/content/drive/My Drive/FakeAVCeleb_v1.2/fvra_features.npy', features)

(1850, 25, 62733)
